In [1]:

import zstandard
import json
from datetime import datetime

import pandas as pd


years = ["2019"]
months = ["07","08","09","10", "11","12"]

for y in years:
    for m in months:
        year_month = f"{y}-{m}"

        non_dep_cohort = pd.read_csv(f"{year_month}-non-dep.csv")
        non_dep_cohort = non_dep_cohort[["author", "created_utc",  "is_self", "subreddit", "subreddit_subscribers", "title", "selftext"]]
        non_dep_cohort = non_dep_cohort.loc[~non_dep_cohort["author"].isin(["[deleted]","AutoModerator"])] 
        non_dep_users = non_dep_cohort["author"].unique()

        print(len(non_dep_users))

        nondep_data = []
        with open(f"RS_{year_month}.zst", 'rb') as fh:
            print(f"RS_{year_month}.zst")
            dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
            with dctx.stream_reader(fh) as reader:
                previous_line = ""
                while True:
                    chunk = reader.read(2**24)  # 16mb chunks
                    if not chunk:
                        break

                    string_data = chunk.decode('utf-8')
                    lines = string_data.split("\n")
                    for i, line in enumerate(lines[:-1]):
                        if i == 0:
                            line = previous_line + line
                        object = json.loads(line)

                        if object["is_self"]:
                            if object["author"] in non_dep_users:
                                nondep_data.append(object)
                    previous_line = lines[-1]
        
        pd.DataFrame(nondep_data).to_csv(f"{year_month}-non-dep-data.csv",index=False)

800
RS_2019-07.zst
877
RS_2019-08.zst
633
RS_2019-09.zst
624
RS_2019-10.zst
606
RS_2019-11.zst
691
RS_2019-12.zst
